In [1]:
#import libraries

import scipy.sparse as sps
import numpy as np
import os
import pandas as pd 

URM_all= sps.load_npz("./Processed Matrices/URM_simple_coo_2020.npz").tocsr()
ICM_all= sps.load_npz("./Processed Matrices/ICM_simple_coo_2020.npz").tocsr()

In [2]:
%load_ext Cython

In [3]:
from Notebooks_utils.evaluation_function import evaluate_algorithm
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

np.random.seed(41148)

In [4]:
from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_FunkSVD_Cython

In [5]:
recommender= MatrixFactorization_FunkSVD_Cython(URM_all)

MatrixFactorization_FunkSVD_Cython_Recommender: URM Detected 1079 (4.15 %) cold items.


In [6]:
recommender.fit(negative_interactions_quota=0.0,
                learning_rate = 0.07213014198205206,
                num_factors=156,
                batch_size=4,
                user_reg=0.01,
                item_reg=1.854523696049507e-05,
                sgd_mode="adagrad", 
                epochs=485)

FUNK_SVD: Processed 113268 ( 100.00% ) in 2.60 seconds. MSE loss 2.37E-01. Sample per second: 43590
FUNK_SVD: Epoch 1 of 485. Elapsed time 1.97 sec
FUNK_SVD: Processed 113268 ( 100.00% ) in 2.25 seconds. MSE loss 3.86E-02. Sample per second: 50367
FUNK_SVD: Epoch 2 of 485. Elapsed time 3.62 sec
FUNK_SVD: Processed 113268 ( 100.00% ) in 1.85 seconds. MSE loss 1.22E-02. Sample per second: 61203
FUNK_SVD: Epoch 3 of 485. Elapsed time 5.23 sec
FUNK_SVD: Processed 113268 ( 100.00% ) in 2.47 seconds. MSE loss 5.16E-03. Sample per second: 45795
FUNK_SVD: Epoch 4 of 485. Elapsed time 6.85 sec
FUNK_SVD: Processed 113268 ( 100.00% ) in 2.05 seconds. MSE loss 2.79E-03. Sample per second: 55168
FUNK_SVD: Epoch 5 of 485. Elapsed time 8.43 sec
FUNK_SVD: Processed 113268 ( 100.00% ) in 1.67 seconds. MSE loss 1.64E-03. Sample per second: 68013
FUNK_SVD: Epoch 6 of 485. Elapsed time 10.04 sec
FUNK_SVD: Processed 113268 ( 100.00% ) in 2.26 seconds. MSE loss 1.07E-03. Sample per second: 50197
FUNK_SVD: E

In [7]:
rec_df=pd.DataFrame({"user_id":[],"item_list":[]})
target_user_list=list(pd.read_csv("./Data/2020/data_target_users_test.csv").user_id)

for uid in target_user_list:
    rec_list=map(str,recommender.recommend(uid)[:10])
    rec_df=rec_df.append({"user_id":uid,"item_list":' '.join(rec_list)},ignore_index=True)

    
rec_df=rec_df.astype({"user_id":'int32',"item_list":'str'})
rec_df=rec_df.set_index('user_id')
rec_df.to_csv("MF_FunkSVD_recommendations.csv")